In [2]:
import pandas as pd
import numpy as np
import re
import math
from tqdm import tqdm
from collections import Counter
from datetime import datetime
from bs4 import BeautifulSoup

## Data Preprocessing

#### 1. Из карточки найти пациентов с ожирением (либо по слову "ожирение", либо по коду МКБ)

In [82]:
def extract_ob_code(value):
    '''Поиск по идентификатору МКБ ожирение: E66
    '''
    re_result = re.findall(r'E66.[0,1,2,8,9]', value)
    if len(re_result) == 0:
        return np.nan
    return re_result


def extract_ob_word(value):
    '''Поиск по словам: "ожирение"
    '''
    re_result = re.findall(r'ожирение', value.lower())
    if len(re_result) == 0:
        return np.nan
    return re_result


def merge_lst(lst):
    '''Функция объединяет списки
    '''
    
    for i in range(len(lst)):
        el = lst[i]
        if pd.isna(el) == False:
            lst[i] = int(el[0])
            
    return lst


def delete_elements(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)


In [3]:
path = 'Obesity/I10_01012015-20210707_all_records.txt'
CHUNK_SIZE = 1000

df_iterator = pd.read_csv(
    path, 
    sep='\t',
    encoding='cp1251',
    chunksize=CHUNK_SIZE)

data = [chunk for chunk in df_iterator]
for i in tqdm(range(len(data))):
    df = data[i]
    data[i] = df[df['RecordEMC'].notna()]

In [6]:
data[0]

,Name,MedicalProcessEventCode,СompletionDate,СompletionTime,SpecialistName,Organization,Department,SpecialistType,EventType,Event,EMCSection,RecordEMC,ExecutionStatus,StartEpizode,EndEpizode,GoalAdmission,TypeAdmission
0,GACAAAY,GACAAAYAAAAbk12:45AAAA,20111206,12:45,Жукова_Ирина_Николаевна,Центр_Алмазова,ООПМУ,Медицинский_регистратор,NaN,Ввод_назначений (Консультация),ИСТОЧНИК_ФИНАНСИРОВАНИЯ,Тип : ДМС Тарифный_план : Все_услуги Плательщи...,выполнено,NaN,20120116.0,NaN,АМБУЛАТОРНО
1,GACAAAY,GACAAAYAAAAbs11:30-cAD,20111214,11:30-12:00,Сапрыкина_Галина_Дмитриевна,Центр_Алмазова,Кар,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ЖАЛОБЫ,"При измерении АД выявил нерегул. ритм с 2008г,...",выполнено,NaN,20120116.0,NaN,АМБУЛАТОРНО
2,GACAAAY,GACAAAYAAAAbs11:30-cAG,20111214,11:30-12:00,Сапрыкина_Галина_Дмитриевна,Центр_Алмазова,Кар,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ЖАЛОБЫ,"АД чаще нормальное-130\60, редко-160\100.",выполнено,NaN,20120116.0,NaN,АМБУЛАТОРНО
3,GACAAAY,GACAAAYAAAAbs11:30-cAJ,20111214,11:30-12:00,Сапрыкина_Галина_Дмитриевна,Центр_Алмазова,Кар,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ЖАЛОБЫ,"Ходит быстро, болей. одышки нет.",выполнено,NaN,20120116.0,NaN,АМБУЛАТОРНО
4,GACAAAY,GACAAAYAAAAbs11:30-cAM,20111214,11:30-12:00,Сапрыкина_Галина_Дмитриевна,Центр_Алмазова,Кар,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ЖАЛОБЫ,"Терапия: конкор 2,5мг, аспирин кардио.",выполнено,NaN,20120116.0,NaN,АМБУЛАТОРНО
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GACAAAY,GACAAAYAAWAAB09:46AuAJ,20160120,09:46,Рой_Алёна_Леонидовна,Центр_Алмазова,КО_№6,Врач-кардиолог,NaN,Курация_пациента_лечащим_врачом,ДНЕВНИК,<w id='GACQK;' > <c id='AC~3;' > Общее состоян...,выполнено,20160119.0,20160122.0,Оперативное_лечение,СТАЦИОНАРНО
996,GACAAAY,GACAAAYAAWAAB09:46AuAM,20160120,09:46,Рой_Алёна_Леонидовна,Центр_Алмазова,КО_№6,Врач-кардиолог,NaN,Курация_пациента_лечащим_врачом,ДНЕВНИК,<w id='GACQK;' > <b> <c id='AD~4;' > Дыхательн...,выполнено,20160119.0,20160122.0,Оперативное_лечение,СТАЦИОНАРНО
997,GACAAAY,GACAAAYAAWAAB09:46AuAP,20160120,09:46,Рой_Алёна_Леонидовна,Центр_Алмазова,КО_№6,Врач-кардиолог,NaN,Курация_пациента_лечащим_врачом,ДНЕВНИК,<w id='GACQK;' > <b> <c id='AE~9;' > Сердечно-...,выполнено,20160119.0,20160122.0,Оперативное_лечение,СТАЦИОНАРНО
998,GACAAAY,GACAAAYAAWAAB09:46AuAS,20160120,09:46,Рой_Алёна_Леонидовна,Центр_Алмазова,КО_№6,Врач-кардиолог,NaN,Курация_пациента_лечащим_врачом,ДНЕВНИК,<w id='GACQK;' > <b> <c id='AF~15;' > Желудочн...,выполнено,20160119.0,20160122.0,Оперативное_лечение,СТАЦИОНАРНО


Important columns:
- Name
- MedicalProcessEventCode
- CompetitionDate
- CompetitionTime
- RecordEMC
- Start Epizode
- EndEpizode

Поиск производится по полю RecordEMC

In [11]:
data[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Name                     1000 non-null   object 
 1   MedicalProcessEventCode  1000 non-null   object 
 2   СompletionDate           1000 non-null   int64  
 3   СompletionTime           1000 non-null   object 
 4   SpecialistName           970 non-null    object 
 5   Organization             1000 non-null   object 
 6   Department               1000 non-null   object 
 7   SpecialistType           970 non-null    object 
 8   EventType                0 non-null      float64
 9   Event                    1000 non-null   object 
 10  EMCSection               1000 non-null   object 
 11  RecordEMC                1000 non-null   object 
 12  ExecutionStatus          1000 non-null   object 
 13  StartEpizode             839 non-null    float64
 14  EndEpizode               

In [19]:
MKB_lst = []

for chunk_id in tqdm(range(len(data))):
    df_tmp = data[chunk_id]
    ind = 0
    for i in df_tmp.index:
        code = extract_ob_code(df_tmp['RecordEMC'][i])
        if code in [np.nan]:
            ind += 1
            continue
        MKB_lst.append(df_tmp.iloc[ind])
        ind += 1

MKB_df = pd.DataFrame(MKB_lst)
MKB_df.to_csv('MKB_list.csv')

100%|████████████████████████████████████| 11329/11329 [01:05<00:00, 173.07it/s]


In [37]:
ob_word_lst = []

for chunk_id in tqdm(range(len(data))):
    df_tmp = data[chunk_id]
    ind = 0
    for i in df_tmp.index:
        code = extract_ob_word(df_tmp['RecordEMC'][i])
        if code in [np.nan]:
            ind += 1
            continue
        ob_word_lst.append(df_tmp.iloc[ind])
        ind += 1

ob_word_df = pd.DataFrame(ob_word_lst)
ob_word_df.to_csv('ob_word_list.csv')

100%|████████████████████████████████████| 11329/11329 [01:20<00:00, 141.55it/s]


In [42]:
ob_word_df

,Name,MedicalProcessEventCode,СompletionDate,СompletionTime,SpecialistName,Organization,Department,SpecialistType,EventType,Event,EMCSection,RecordEMC,ExecutionStatus,StartEpizode,EndEpizode,GoalAdmission,TypeAdmission
4120,GACAACH,GACAACHABDAAA11:16BmAP,20160113,11:16,Орлова_Ольга_Владимировна,Центр_Алмазова,Кар,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ОБЩИЙ_ОСМОТР,<w id='GACAI;' > <c id='AI~11;' > Состояние пи...,выполнено,NaN,20160113.0,NaN,АМБУЛАТОРНО
4215,GACAACH,GACAACHABFAAANABИaAkAS,20180206,10:28,Герасименко_Ольга_Владимировна,Центр_Алмазова,Кар,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ОБЩИЙ_ОСМОТР,<w id='GACAI;' > <c id='AI~11;' > Состояние пи...,выполнено,NaN,20180219.0,NaN,АМБУЛАТОРНО
4255,GACAACH,GACAACHABHAAA12:52AeAD,20180206,12:52,Глебовская_Татьяна_Дмитриевна,Центр_Алмазова,ОА-Р_№6_(КПК),Заведующий_отделением_анестезиологии_и_реанима...,NaN,Суточное_наблюдение_врача-реаниматолога,АНАМНЕЗ_ЖИЗНИ,<w id='GACAB;' > <b> <c id='Ah~25;' > Хроничес...,выполнено,20180206.0,20180220.0,Лечебно-диагностическая,СТАЦИОНАРНО
4667,GACAACH,GACAACHABHAAG12:10AgAD,20180212,12:10,Калачева_Анастасия_Павловна,Центр_Алмазова,КО_№5_(Пархоменко),Клинический_ординатор,NaN,Осмотр_клиническим_ординатором,АНАМНЕЗ_ЖИЗНИ,<w id='GACAu;' > <b> <c id='AG~25;' > Хроничес...,выполнено,20180206.0,20180220.0,Лечебно-диагностическая,СТАЦИОНАРНО
4678,GACAACH,GACAACHABHAAG12:10AkAS,20180212,12:10,Калачева_Анастасия_Павловна,Центр_Алмазова,КО_№5_(Пархоменко),Клинический_ординатор,NaN,Осмотр_клиническим_ординатором,ОБЩИЙ_ОСМОТР,<w id='GACRI;' > <c id='AH~10;' > Состояние пи...,выполнено,20180206.0,20180220.0,Лечебно-диагностическая,СТАЦИОНАРНО
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11323425,GACBB6F,GACBB6FAAJAAA18:22A!AE,20210426,18:22,Дикарева_Елена_Леонтьевна,Центр_Алмазова,ОПБ,Врач-акушер-гинеколог,NaN,Курация_пациента_лечащим_врачом,ПЕРВИЧНЫЙ_ОСМОТР_АКУШЕРА-ГИНЕКОЛОГА_,<w id='GACMT;' > <p style='text-align:center;'...,выполнено,20210426.0,20210428.0,Оперативное_лечение,СТАЦИОНАРНО
11323719,GACBB6F,GACBB6FAAQAAB20:26A!AB,20210513,20:26,Гаджиева_Жавгарат_Ахмедовна,Центр_Алмазова,ОПБ,Клинический_ординатор,NaN,Осмотр_клиническим_ординатором,ПЕРВИЧНЫЙ_ОСМОТР_АКУШЕРА-ГИНЕКОЛОГА_,<w id='GACMT;' > <v id='AAAA~166;' > с_заведую...,выполнено,20210512.0,20210526.0,Оперативное_лечение,СТАЦИОНАРНО
11323798,GACBB6F,GACBB6FAAQAAFNAAзYA%AB,20210517,09:44,Володичева_Наталия_Сергеевна,Центр_Алмазова,ОКП,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",АНАМНЕЗ_ЖИЗНИ,<w id='GACAu;' > <b> <c id='AG~25;' > Хроничес...,выполнено,20210512.0,20210526.0,Оперативное_лечение,СТАЦИОНАРНО
11323804,GACBB6F,GACBB6FAAQAAFNAAзYA'AF,20210517,09:44,Володичева_Наталия_Сергеевна,Центр_Алмазова,ОКП,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ОБЩИЙ_ОСМОТР,<w id='GACAI;' > <c id='AJ~11;' > Состояние пи...,выполнено,20210512.0,20210526.0,Оперативное_лечение,СТАЦИОНАРНО


Read data from files: MKB_list.csv and ob_word_list.csv

Merge them

In [19]:
data_MKB = pd.read_csv('MKB_list.csv', index_col=0)
data_ob_word = pd.read_csv('ob_word_list.csv', index_col=0)

ind1 = data_MKB.index
ind2 = data_ob_word.index

duplicates = list(set(ind1).intersection(ind2))
data_ob_word = data_ob_word.drop(duplicates)
data = pd.concat([data_ob_word, data_MKB])

data.to_csv("dataset.csv")

#### 2. Извлечем из dataset взвешивания 

Important columns:
- Name
- MedicalProcessEventCode (в датасете с анализами аналогичный столбец называется "epizod")
- RecordEMC
- StartEpizode
- EndEpizode

In [306]:
dataset = pd.read_csv("dataset.csv", index_col=0)
col = ["Name", "MedicalProcessEventCode", "RecordEMC", "StartEpizode"]
dataset = dataset[col]
dataset

,Name,MedicalProcessEventCode,RecordEMC,StartEpizode
4120,GACAACH,GACAACHABDAAA11:16BmAP,<w id='GACAI;' > <c id='AI~11;' > Состояние пи...,NaN
4215,GACAACH,GACAACHABFAAANABИaAkAS,<w id='GACAI;' > <c id='AI~11;' > Состояние пи...,NaN
4255,GACAACH,GACAACHABHAAA12:52AeAD,<w id='GACAB;' > <b> <c id='Ah~25;' > Хроничес...,20180206.0
4667,GACAACH,GACAACHABHAAG12:10AgAD,<w id='GACAu;' > <b> <c id='AG~25;' > Хроничес...,20180206.0
4678,GACAACH,GACAACHABHAAG12:10AkAS,<w id='GACRI;' > <c id='AH~10;' > Состояние пи...,20180206.0
...,...,...,...,...
11309984,GACBAкЪ,GACBAкЪAAWAAP17:38A!AA,<b>КЛИНИЧЕСКИЙ ДИАГНОЗ:</b><br/><br/><b>_ОСНОВ...,20201013.0
11309990,GACBAкЪ,GACBAкЪAAWAAP17:38A#AG,<w id='#ABVw;' > <c id='AO~34;' > Данные о МЭС...,20201013.0
11310216,GACBAкЪ,GACBAкЪAAaAAA12:40A!AA,<b>КЛИНИЧЕСКИЙ ДИАГНОЗ:</b><br/><b>Основной:</...,NaN
11317277,GACBAэы,GACBAэыAAEAAG10:58A(AB,<b>_СОПУТСТВУЮЩИЙ КЛИНИЧЕСКИЙ_ : </b><b>Код по...,20210407.0


Найдем пациентов, у которых есть измерения веса, сохраним данные записи в датасет time_weight_df

Столбцы:  Name, MedicalProcessEventCode, StartEpizode, EndEpizode


In [307]:
time_weight_lst = []
tmp_ind = dataset.columns.tolist().index("RecordEMC")

for i in tqdm(range(len(dataset))):
    record = dataset.iloc[i, tmp_ind]
    
    if re.search(r"Вес", record):
        time_weight_lst.append(dataset.iloc[i, :])
        
time_weight_col = ["Name", "MedicalProcessEventCode", "RecordEMC", "StartEpizode"]        
time_weight_df = pd.DataFrame(time_weight_lst, columns=time_weight_col)
time_weight_df

100%|██████████████████████████████████| 48284/48284 [00:02<00:00, 17433.85it/s]


,Name,MedicalProcessEventCode,RecordEMC,StartEpizode
4120,GACAACH,GACAACHABDAAA11:16BmAP,<w id='GACAI;' > <c id='AI~11;' > Состояние пи...,NaN
4215,GACAACH,GACAACHABFAAANABИaAkAS,<w id='GACAI;' > <c id='AI~11;' > Состояние пи...,NaN
4678,GACAACH,GACAACHABHAAG12:10AkAS,<w id='GACRI;' > <c id='AH~10;' > Состояние пи...,20180206.0
5477,GACAACH,GACAACHABVAAA12:47A%AP,<w id='GACAI;' > <c id='AJ~11;' > Состояние пи...,NaN
5843,GACAACh,GACAAChAABAKo11:30-kAV,Состояние питания удовлетворительное : Вес 82 ...,20110412.0
...,...,...,...,...
11323719,GACBB6F,GACBB6FAAQAAB20:26A!AB,<w id='GACMT;' > <v id='AAAA~166;' > с_заведую...,20210512.0
11323804,GACBB6F,GACBB6FAAQAAFNAAзYA'AF,<w id='GACAI;' > <c id='AJ~11;' > Состояние пи...,20210512.0
8288631,GACAYтЫ,GACAYтЫAANAFm160016cAD,<w id='GACAh;' > <v id='AA~1;' val=' Пациент н...,20170407.0
9658930,GACAar@,GACAar@AADAAN17:28A&AA,"Черкашин Никита Владимирович, дата рождения 13...",20180221.0


In [308]:
# удалим записи, у которых Startepizode == NaN
time_weight_df.dropna(subset = ["StartEpizode"], inplace=True)

In [309]:
# удалим пациентов с одним измерением веса
patient_ids = time_weight_df["Name"].unique()

for patient in tqdm(patient_ids):
    # indexes of rows with the value of the column Name = patient
    tmp_ind = time_weight_df.index[time_weight_df['Name'] == patient].tolist()
    if len(tmp_ind) == 1:
        time_weight_df = time_weight_df.drop(tmp_ind)

100%|██████████████████████████████████████| 6129/6129 [00:10<00:00, 596.24it/s]


In [310]:
# преборазуем дату float->date()
for i in time_weight_df.index:
    date = re.findall(r"(\d{4})(\d{2})(\d{2})", str(int(time_weight_df["StartEpizode"][i])))
    date = '-'.join(date[0])
    date = datetime.strptime(date, "%Y-%m-%d").date()
    
    time_weight_df["StartEpizode"][i] = date
    

/tmp/ipykernel_24556/4159366009.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_weight_df["StartEpizode"][i] = date


In [311]:
time_weight_df

,Name,MedicalProcessEventCode,RecordEMC,StartEpizode
9544,GACAAHY,GACAAHYAACAAC11:30-kAP,Состояние питания удовлетворительное : Вес 75 ...,2012-09-08
9612,GACAAHY,GACAAHYAAFAAA13:47AkAY,Состояние питания удовлетворительное : Вес 78 ...,2012-11-02
9964,GACAAHe,GACAAHeAACABhN20739iAD,"Телосложение: гиперстеник Вес, кг : 110 Рост...",2011-04-19
10066,GACAAHe,GACAAHeAADAFU11:30-kAV,Состояние питания удовлетворительное : Вес 105...,2011-09-22
10162,GACAAHe,GACAAHeAAFAAC09:00-kAV,Состояние питания удовлетворительное : Вес 102...,2013-04-09
...,...,...,...,...
11323719,GACBB6F,GACBB6FAAQAAB20:26A!AB,<w id='GACMT;' > <v id='AAAA~166;' > с_заведую...,2021-05-12
11323804,GACBB6F,GACBB6FAAQAAFNAAзYA'AF,<w id='GACAI;' > <c id='AJ~11;' > Состояние пи...,2021-05-12
8288631,GACAYтЫ,GACAYтЫAANAFm160016cAD,<w id='GACAh;' > <v id='AA~1;' val=' Пациент н...,2017-04-07
9658930,GACAar@,GACAar@AADAAN17:28A&AA,"Черкашин Никита Владимирович, дата рождения 13...",2018-02-21


1. Для каждого пациента найти первое взвешивание и взвешивание, расстояние между которым и первоначальным больше месяца, но меньше года, промежуточные удалить


2. Посчитать разность между взвешиваниями (не меньше месяца, не больше года), удалить тех пациентов, 
которые не удовлетворяют условию

3. Извлечь первоначальный и конечный веc

1-3 сохранять в weight_time_df (далее пригодится)

В новый датасет извлечь Name, MedicalProcessEventCode, Weight (init), период между взвешиваниями

Определить пациент набрал или сбросил вес, записать в новый датасет (predictor)

По Name извлечь из анализов всех инфу + проверить по датам (все индикаторы, ближайщие к первому взвешиванию,
либо до него)


In [312]:
# разберем алгоритм для одного пациента
patient_ids_lst = []

patient_df = time_weight_df[time_weight_df["Name"]=="GACBAкЪ"]
patient_df = patient_df.sort_values(by=['StartEpizode'])

time_lst = patient_df["StartEpizode"].tolist()
print(time_lst)
# условие: запустить цикл, пока разница между первой датой и текущей меньше года (365),
# как только достигли конца цикла, приравниваем текущее значение ко второму измерению
# проверяем, больше ли разница месяца (30)
date_start = time_lst[0]
date_next = time_lst[1]

diff = date_next - date_start
print(diff)
print(diff.days > 30)
print(diff.days < 365)

for i in range(len(time_lst)):
    diff = time_lst[i] - date_start
    
    if diff.days > 365:
        date_next = time_lst[i-1]
        break
    
    date_next = time_lst[i]
    
diff = date_next - date_start
print(diff)
if diff.days >= 30:
    # берем id пациента
    # сохраняем в отдельный датасет имя, date_init, date_next
    patient_ids_lst.append(["GACBAкЪ", date_start, date_next])

print(patient_ids_lst)

patient_ids_df = pd.DataFrame(patient_ids_lst, columns=["Name", "DateStart", "DateNext"])
patient_ids_df

[datetime.date(2019, 11, 22), datetime.date(2019, 12, 6), datetime.date(2020, 2, 6), datetime.date(2020, 7, 30), datetime.date(2020, 10, 13), datetime.date(2020, 10, 13), datetime.date(2020, 10, 13), datetime.date(2020, 10, 13), datetime.date(2020, 10, 13), datetime.date(2020, 10, 13)]
14 days, 0:00:00
False
True
326 days, 0:00:00
[['GACBAкЪ', datetime.date(2019, 11, 22), datetime.date(2020, 10, 13)]]


,Name,DateStart,DateNext
0,GACBAкЪ,2019-11-22,2020-10-13


In [313]:
# реализуем алгоритм для всех пациентов
patient_ids = time_weight_df["Name"].unique()
patient_ids_lst = []
start_epizode_lst = []

for name in patient_ids:
    patient_df = time_weight_df[time_weight_df["Name"]==name]
    patient_df = patient_df.sort_values(by=['StartEpizode'])
    
    time_lst = patient_df["StartEpizode"].tolist()
    date_start = time_lst[0]
    # условие: запустить цикл, пока разница между первой датой и текущей меньше года (365),
    # как только достигли конца цикла, приравниваем текущее значение ко второму измерению
    # проверяем, больше ли разница месяца (30)
    for i in range(len(time_lst)):
        diff = time_lst[i] - date_start
    
        if diff.days > 365:
            date_next = time_lst[i-1]
            break
        
        date_next = time_lst[i]

    diff = date_next - date_start

    if diff.days >= 30:
        # берем id пациента
        # сохраняем в отдельный датасет имя, date_init, date_next
        start_id = patient_df[patient_df["StartEpizode"]==date_start].index
        
        tmp_lst = []
        for i in start_id:
            tmp_lst.append(patient_df.loc[i, "MedicalProcessEventCode"])
        
        start_epizode_lst.append(tmp_lst)
        
        start_id = patient_df[patient_df["StartEpizode"]==date_start].index[0]
        next_id = patient_df[patient_df["StartEpizode"]==date_next].index[0]
       
        patient_ids_lst.append(patient_df.loc[start_id].values.flatten().tolist())
        patient_ids_lst.append(patient_df.loc[next_id].values.flatten().tolist())

In [314]:
patient_ids_df = pd.DataFrame(patient_ids_lst, columns=patient_df.columns)
patient_ids_df

,Name,MedicalProcessEventCode,RecordEMC,StartEpizode
0,GACAAHY,GACAAHYAACAAC11:30-kAP,Состояние питания удовлетворительное : Вес 75 ...,2012-09-08
1,GACAAHY,GACAAHYAAFAAA13:47AkAY,Состояние питания удовлетворительное : Вес 78 ...,2012-11-02
2,GACAAHe,GACAAHeAACABhN20739iAD,"Телосложение: гиперстеник Вес, кг : 110 Рост...",2011-04-19
3,GACAAHe,GACAAHeAADAFU11:30-kAV,Состояние питания удовлетворительное : Вес 105...,2011-09-22
4,GACAAKt,GACAAKtAABAAIN21675kAP,Состояние питания ожирение : Вес 100 <sub> кг ...,2013-05-20
...,...,...,...,...
1613,GACBAкЪ,GACBAкЪAAWAAA16:41A'AH,<w id='GACAI;' red='AJ;' > Состояние питания у...,2020-10-13
1614,GACBAэы,GACBAэыAADAAA152016&AH,<w id='GACCi;' > <c id='AG~7;' > Состояние пит...,2020-11-16
1615,GACBAэы,GACBAэыAAEAAA13:52A'AG,<w id='GACAI;' > <c id='AJ~11;' > Состояние пи...,2021-04-07
1616,GACBB6F,GACBB6FAAEAAC09:52A!AD,<w id='GACMT;' > <p style='text-align:center;'...,2021-03-13


In [315]:
patient_ids = patient_ids_df["Name"].unique()
event_code_lst = [list(tup) for tup in zip(patient_ids, start_epizode_lst)]
event_code_df = pd.DataFrame(event_code_lst, columns=["Name", "MedicalProcessEventCode"])
event_code_df.to_csv("event_code.csv")

Выделить вес:

- очистить данные 

- выделить вес и рост

- найти разницу между измерениями веса

In [316]:
# Очистим текст: теги, единицы измерения, знаки препинания
tmp_ind = patient_ids_df.columns.tolist().index("RecordEMC")

marks = '''!()-[]{};?@#$%:'"\,/^&amp;*_'''
units = ['кг', 'см']

for i in patient_ids_df.index:
    record = patient_ids_df.iloc[i, tmp_ind]
    
    # удаление HTML-тегов
    soup = BeautifulSoup(record)
    text = soup.get_text()
    
    # удаление знаков препинания
    for x in text:  
        if x in marks:  
            text = text.replace(x, "")  
    
    # удаление единиц измерения
    text = re.sub(re.compile(
            r"(\s+(кг|см))"), ' ', text)
    
    # удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text)
    
    patient_ids_df.iloc[i, tmp_ind] = text.lower()


In [323]:
patient_ids_df["RecordEMC"][105]

'состояние питания удовлетворительное вес 80 рост 155 индекс массы тела 33.30 ожирение i площадь поверхности тела 1.86'

In [318]:
# Извлечем вес и рост

records = patient_ids_df["RecordEMC"].tolist()

weight_lst = []
height_lst = []
tmp_ind = patient_ids_df.columns.tolist().index("RecordEMC")

for i in range(len(patient_ids_df)):
    record = patient_ids_df.iloc[i, tmp_ind]
    
    weight = re.findall(r"вес\s(\d{3}).*рост", record)
    if len(weight) == 0:
        weight = re.findall(r"вес\s(\d{2}).*рост", record)
        
    height = re.findall(r"рост\s(\d{3})[\.\s]", record)
    if len(height) == 0:
        height = re.findall(r"рост\s(\d{2})[\.\s]", record)
        
    if weight:
        weight_lst.append(weight)
    else:
        weight_lst.append(np.nan)
    
    if height:
        height_lst.append(height)
    else:
        height_lst.append(np.nan)

In [319]:
weight_lst = merge_lst(weight_lst)
height_lst = merge_lst(height_lst)
patient_ids_df["Weight"], patient_ids_df["Height"] = weight_lst, height_lst

In [338]:
# Извлечем степень ожирения:
# i, ii, iii
records = patient_ids_df["RecordEMC"].tolist()
ob_d = []
tmp_ind = patient_ids_df.columns.tolist().index("RecordEMC")

for i in range(len(patient_ids_df)):
    record = patient_ids_df.iloc[i, tmp_ind]
    ob = re.findall(r"ожирение (i{1,3})", record)
    if ob:
        ob_d.append(len(ob[0]))
    else:
        ob_d.append(np.nan)

In [339]:
patient_ids_df["ObesityDegree"] = ob_d

In [340]:
patient_ids_df

,Name,MedicalProcessEventCode,RecordEMC,StartEpizode,Weight,Height,ObesityDegree
0,GACAAHY,GACAAHYAACAAC11:30-kAP,состояние питания удовлетворительное вес 75 ро...,2012-09-08,75.0,150.0,1.0
1,GACAAHY,GACAAHYAAFAAA13:47AkAY,состояние питания удовлетворительное вес 78 ро...,2012-11-02,78.0,150.0,1.0
2,GACAAHe,GACAAHeAACABhN20739iAD,телосложение гиперстеник вес 110 рост 153 инде...,2011-04-19,110.0,153.0,3.0
3,GACAAHe,GACAAHeAADAFU11:30-kAV,состояние питания удовлетворительное вес 105 р...,2011-09-22,105.0,163.0,2.0
4,GACAAKt,GACAAKtAABAAIN21675kAP,состояние питания ожирение вес 100 рост 176 ин...,2013-05-20,100.0,176.0,1.0
...,...,...,...,...,...,...,...
1613,GACBAкЪ,GACBAкЪAAWAAA16:41A'AH,состояние питания удовлетворительное вес 102....,2020-10-13,102.0,181.0,1.0
1614,GACBAэы,GACBAэыAADAAA152016&AH,состояние питания удовлетворительное вес 130....,2020-11-16,130.0,190.0,2.0
1615,GACBAэы,GACBAэыAAEAAA13:52A'AG,состояние питания ожирение вес 134.3 рост 188...,2021-04-07,134.0,188.0,2.0
1616,GACBB6F,GACBB6FAAEAAC09:52A!AD,объективный статус общее состояние удовлетвор...,2021-03-13,120.0,158.0,3.0


In [341]:
# remove nan
# удалить пациентов с одним измерением веса

patient_ids_df = patient_ids_df.dropna(axis='index', how='any', subset=['Weight'])
patient_ids_df.reset_index(drop=True, inplace=True)

# удалим пациентов с одним измерением веса
patient_ids = patient_ids_df["Name"].unique()

for patient in tqdm(patient_ids):
    # indexes of rows with the value of the column Name = patient
    tmp_ind = patient_ids_df.index[patient_ids_df['Name'] == patient].tolist()
    if len(tmp_ind) == 1:
        patient_ids_df = patient_ids_df.drop(tmp_ind)

patient_ids_df.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████| 802/802 [00:01<00:00, 796.42it/s]


In [342]:
patient_ids_df

,Name,MedicalProcessEventCode,RecordEMC,StartEpizode,Weight,Height,ObesityDegree
0,GACAAHY,GACAAHYAACAAC11:30-kAP,состояние питания удовлетворительное вес 75 ро...,2012-09-08,75.0,150.0,1.0
1,GACAAHY,GACAAHYAAFAAA13:47AkAY,состояние питания удовлетворительное вес 78 ро...,2012-11-02,78.0,150.0,1.0
2,GACAAHe,GACAAHeAACABhN20739iAD,телосложение гиперстеник вес 110 рост 153 инде...,2011-04-19,110.0,153.0,3.0
3,GACAAHe,GACAAHeAADAFU11:30-kAV,состояние питания удовлетворительное вес 105 р...,2011-09-22,105.0,163.0,2.0
4,GACAAKt,GACAAKtAABAAIN21675kAP,состояние питания ожирение вес 100 рост 176 ин...,2013-05-20,100.0,176.0,1.0
...,...,...,...,...,...,...,...
1581,GACBAкЪ,GACBAкЪAAWAAA16:41A'AH,состояние питания удовлетворительное вес 102....,2020-10-13,102.0,181.0,1.0
1582,GACBAэы,GACBAэыAADAAA152016&AH,состояние питания удовлетворительное вес 130....,2020-11-16,130.0,190.0,2.0
1583,GACBAэы,GACBAэыAAEAAA13:52A'AG,состояние питания ожирение вес 134.3 рост 188...,2021-04-07,134.0,188.0,2.0
1584,GACBB6F,GACBB6FAAEAAC09:52A!AD,объективный статус общее состояние удовлетвор...,2021-03-13,120.0,158.0,3.0


In [344]:
patient_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586 entries, 0 to 1585
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Name                     1586 non-null   object 
 1   MedicalProcessEventCode  1586 non-null   object 
 2   RecordEMC                1586 non-null   object 
 3   StartEpizode             1586 non-null   object 
 4   Weight                   1586 non-null   float64
 5   Height                   1580 non-null   float64
 6   ObesityDegree            1569 non-null   float64
dtypes: float64(3), object(4)
memory usage: 86.9+ KB


#### 3. Сформируем target (потеря, либо набор веса)

В датасет добавляем name, weight_init, height (в начальной точке), date_diff, target, weight_diff, target_weight, StartEpizode

Считаем: 
- разницу в весе - target (-1 - потеря, 0 - сохранение веса, 1 - набор), сохранить разницу 
- разницу в дате (date diff)


In [349]:
patient_ids = patient_ids_df["Name"].unique()
index = 0
weight_init_lst = []
diff_weight_lst = []
diff_date_lst = []
target = []
height_lst = []
target_weight = []
start_epizode_lst = []
ob_d_lst = []

for patient in patient_ids:
    tmp_df = patient_ids_df[patient_ids_df["Name"]==patient]
    weight_init, weight_next = tmp_df["Weight"][index], tmp_df["Weight"][index+1]
    weight_diff = weight_next - weight_init
    date_diff = tmp_df["StartEpizode"][index+1]-tmp_df["StartEpizode"][index]
    start_epizode = tmp_df["StartEpizode"][index]
    ob_d = tmp_df["ObesityDegree"][index]
    
    if weight_diff < 0:
        t = -1
    elif weight_diff > 0:
        t = 1
    else:
        t = 0
    
    target.append(t)
    height_lst.append((tmp_df["Height"][index]+tmp_df["Height"][index+1])/2)
    diff_date_lst.append(date_diff.days)
    diff_weight_lst.append(weight_diff)
    weight_init_lst.append(weight_init)
    target_weight.append(weight_next)
    start_epizode_lst.append(start_epizode)
    ob_d_lst.append(ob_d)
    
    index += 2

In [350]:
dataset_target = pd.DataFrame(columns = ["Name", "WeightInit", "Height", "DateDiff", 
                                         "WeightDiff", "Target", "TargetWeight", "StartEpizode"])

dataset_target["Name"] = patient_ids
dataset_target["WeightInit"] = weight_init_lst
dataset_target["Height"] = height_lst
dataset_target["DateDiff"] = diff_date_lst
dataset_target["WeightDiff"] = diff_weight_lst
dataset_target["Target"] = target
dataset_target["TargetWeight"] = target_weight
dataset_target["StartEpizode"] = start_epizode_lst
dataset_target["ObesityDegree"] = ob_d_lst

In [351]:
dataset_target

,Name,WeightInit,Height,DateDiff,WeightDiff,Target,TargetWeight,StartEpizode,ObesityDegree
0,GACAAHY,75.0,150.0,55,3.0,1,78.0,2012-09-08,1.0
1,GACAAHe,110.0,158.0,156,-5.0,-1,105.0,2011-04-19,3.0
2,GACAAKt,100.0,176.0,32,0.0,0,100.0,2013-05-20,1.0
3,GACAAVB,97.0,168.5,292,1.0,1,98.0,2015-01-24,1.0
4,GACAAXQ,100.0,172.5,358,0.0,0,100.0,2018-01-23,1.0
...,...,...,...,...,...,...,...,...,...
788,GACBAНg,100.0,168.0,317,2.0,1,102.0,2017-06-20,2.0
789,GACBAХH,98.0,174.0,233,-6.0,-1,92.0,2019-04-24,1.0
790,GACBAкЪ,116.0,182.0,326,-14.0,-1,102.0,2019-11-22,1.0
791,GACBAэы,130.0,189.0,142,4.0,1,134.0,2020-11-16,2.0


In [352]:
dataset_target.to_csv("dataset_target.csv")

## Extract data from ChangedWeightObesity.csv

In [3]:
data = pd.read_csv("Obesity/ChangedWeightObesity.csv", index_col=0)

In [31]:
data

,Name,MedicalProcessEventCode,СompletionDate,СompletionTime,SpecialistName,Organization,Department,SpecialistType,EventType,Event,EMCSection,RecordEMC,ExecutionStatus,StartEpizode,EndEpizode,GoalAdmission,TypeAdmission
4120,GACAACH,GACAACHABDAAA11:16BmAP,20160113.0,11:16,Орлова_Ольга_Владимировна,Центр_Алмазова,Кар,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ОБЩИЙ_ОСМОТР,<w id='GACAI;' > <c id='AI~11;' > Состояние пи...,выполнено,NaN,20160113.0,NaN,АМБУЛАТОРНО
4215,GACAACH,GACAACHABFAAANABИaAkAS,20180206.0,10:28,Герасименко_Ольга_Владимировна,Центр_Алмазова,Кар,Врач-кардиолог,NaN,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ОБЩИЙ_ОСМОТР,<w id='GACAI;' > <c id='AI~11;' > Состояние пи...,выполнено,NaN,20180219.0,NaN,АМБУЛАТОРНО
4255,GACAACH,GACAACHABHAAA12:52AeAD,20180206.0,12:52,Глебовская_Татьяна_Дмитриевна,Центр_Алмазова,ОА-Р_№6_(КПК),Заведующий_отделением_анестезиологии_и_реанима...,NaN,Суточное_наблюдение_врача-реаниматолога,АНАМНЕЗ_ЖИЗНИ,<w id='GACAB;' > <b> <c id='Ah~25;' > Хроничес...,выполнено,20180206.0,20180220.0,Лечебно-диагностическая,СТАЦИОНАРНО
4667,GACAACH,GACAACHABHAAG12:10AgAD,20180212.0,12:10,Калачева_Анастасия_Павловна,Центр_Алмазова,КО_№5_(Пархоменко),Клинический_ординатор,NaN,Осмотр_клиническим_ординатором,АНАМНЕЗ_ЖИЗНИ,<w id='GACAu;' > <b> <c id='AG~25;' > Хроничес...,выполнено,20180206.0,20180220.0,Лечебно-диагностическая,СТАЦИОНАРНО
4678,GACAACH,GACAACHABHAAG12:10AkAS,20180212.0,12:10,Калачева_Анастасия_Павловна,Центр_Алмазова,КО_№5_(Пархоменко),Клинический_ординатор,NaN,Осмотр_клиническим_ординатором,ОБЩИЙ_ОСМОТР,<w id='GACRI;' > <c id='AH~10;' > Состояние пи...,выполнено,20180206.0,20180220.0,Лечебно-диагностическая,СТАЦИОНАРНО
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11309984,GACBAкЪ,GACBAкЪAAWAAP17:38A!AA,20201028.0,17:38,Лискина_Анастасия_Сергеевна,Центр_Алмазова,ОПиМРдет_№2_ЛРКдет,Врач-детский_эндокринолог,NaN,Выписка_из_стационара,ДИАГНОЗ_СТАЦИОНАРНЫЙ,<b>КЛИНИЧЕСКИЙ ДИАГНОЗ:</b><br/><br/><b>_ОСНОВ...,выполнено,20201013.0,20201028.0,Лечебно-диагностическая,СТАЦИОНАРНО
11309990,GACBAкЪ,GACBAкЪAAWAAP17:38A#AG,20201028.0,17:38,Лискина_Анастасия_Сергеевна,Центр_Алмазова,ОПиМРдет_№2_ЛРКдет,Врач-детский_эндокринолог,NaN,Выписка_из_стационара,ЗАКРЫТИЕ_ЭПИЗОДА,<w id='#ABVw;' > <c id='AO~34;' > Данные о МЭС...,выполнено,20201013.0,20201028.0,Лечебно-диагностическая,СТАЦИОНАРНО
11310216,GACBAкЪ,GACBAкЪAAaAAA12:40A!AA,20201015.0,12:40,Плаксина_Анна_Олеговна,Центр_Алмазова,ОПиМРдет_№2_ЛРКдет,Врач-детский_эндокринолог,NaN,Регистрация_пациента_по_специализированной_мед...,ДИАГНОЗ_АМБУЛАТОРНЫЙ,<b>КЛИНИЧЕСКИЙ ДИАГНОЗ:</b><br/><b>Основной:</...,выполнено,NaN,NaN,NaN,СМП
11317277,GACBAэы,GACBAэыAAEAAG10:58A(AB,20210413.0,10:58,Эртман_Александр_Энгельсович,Центр_Алмазова,ЛРК2КО_№9,Врач-кардиолог,NaN,Курация_пациента_лечащим_врачом,ДИАГНОЗ_СТАЦИОНАРНЫЙ,<b>_СОПУТСТВУЮЩИЙ КЛИНИЧЕСКИЙ_ : </b><b>Код по...,выполнено,20210407.0,20210416.0,Лечебно-диагностическая,СТАЦИОНАРНО


In [5]:
data.columns

Index(['Рост', 'Вес', 'сибутрамин', 'АГ', 'i10', 'ХСН', 'ИМ', 'Анемия',
       'Острые_легочные_осложнения', 'Метформин', 'Кортикостероиды', 'ИБС',
       'Вредный_образ_жизни', 'ФП', 'НТГ', 'Ожирение', 'Cтенокардия',
       'Расстройства_сна', 'Нарушения_обмена_липопротеидов', 'Гипергликемия',
       'нейропатия', 'диабетическая_ангиопатия', 'инсульт', 'Курение',
       'Алкоголь', 'Гиподинамия', 'Депрессия', 'Тахикардия', 'Ангиография',
       'Нитрат', 'Желудочковая_аритмия', 'Диабет', 'витамин',
       'Антигипоксанты и антиоксиданты', 'Sex', 'atherosclerosis', 'Age',
       'ИМТ', 'ППТ', 'target_Вес', 'степень_ожирения'],
      dtype='object')

## Extract data from AllPatientInformation20210707.txt

In [356]:
path = "Obesity/Analisis/AllPatientInformation20210707.txt"

data = pd.read_csv(path, sep='\t', encoding='cp1251')
data

/tmp/ipykernel_24556/1564887385.py:3: DtypeWarning: Columns (1,2,3,4,7,8,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path, sep='\t', encoding='cp1251')


,PatientName,Bithday,Gender,BloodGroup,Rezus,InvGroup,NumberContract,Department0,Department1,Department2,Department3,FIOUl,TypeHospital,SendingOrganization
0,#AAAAAA,19510409,Женский,NaN,NaN,NaN,1/09,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
1,#AAAAAB,NaN,Женский,NaN,NaN,NaN,1/A11,NaN,NaN,NaN,СП.АРМ,NaN,NaN,NaN
2,#AAAAAC,NaN,Женский,NaN,NaN,NaN,4/A10,NaN,NaN,NaN,СП.АРМ,NaN,NaN,NaN
3,#AAAAAD,NaN,Женский,NaN,NaN,NaN,3/A12,NaN,NaN,NaN,СП.АРМ,NaN,NaN,NaN
4,#AAAAAE,NaN,Женский,NaN,NaN,NaN,4/A12,NaN,NaN,NaN,СП.АРМ,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065832,yAAlAC=,NaN,NaN,NaN,NaN,NaN,l305,NaN,NaN,l307,МИАЦ_СПб,ХА ХА,ХА ХА,NaN
1065833,yAraAAA,NaN,NaN,NaN,NaN,NaN,ayAr,NaN,Организация_МИАЦ_(ОДЛИ),МИАЦ_(ОДЛИ),МИАЦ_(ОДЛИ),ОРГАНИЗАЦИЯ МИАЦ ОДЛИ,ОРГАНИЗАЦИЯ МИАЦ ОДЛИ,NaN
1065834,яяя~GAC,NaN,NaN,NaN,NaN,NaN,/212,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1065835,яяяАМБУ,NaN,Мужской,NaN,NaN,NaN,1/А13,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [357]:
dataset = pd.read_csv("dataset_target.csv", index_col=0)
patient_ids_target = dataset["Name"]
patient_lst = []

for patient in tqdm(patient_ids_target):
    patient_lst.append(data[data["PatientName"]==patient].values.tolist()[0])
#     if patient not in patient_ids_target:
#         tmp_ind = data.index[data['PatientName'] == patient].tolist()
#         data = data.drop(tmp_ind)
        
# data.reset_index(drop=True, inplace=True)

100%|█████████████████████████████████████████| 793/793 [00:46<00:00, 17.13it/s]


In [358]:
col = data.columns
data = pd.DataFrame(patient_lst, columns=col)
data

,PatientName,Bithday,Gender,BloodGroup,Rezus,InvGroup,NumberContract,Department0,Department1,Department2,Department3,FIOUl,TypeHospital,SendingOrganization
0,GACAAHY,19470129,Женский,NaN,NaN,NaN,400/A07,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
1,GACAAHe,19461001,Женский,NaN,NaN,NaN,406/A07,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
2,GACAAKt,19490409,Мужской,NaN,NaN,NaN,577/A07,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
3,GACAAVB,19650323,Мужской,NaN,NaN,NaN,491/A08,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
4,GACAAXQ,19510224,Мужской,NaN,NaN,NaN,610/A08,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,GACBAНg,19731118.0,Женский,NaN,NaN,NaN,90266/B18,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
789,GACBAХH,19580416.0,Женский,NaN,NaN,NaN,16503/B19,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
790,GACBAкЪ,20030724.0,Мужской,NaN,NaN,NaN,91027/B19,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
791,GACBAэы,19970805.0,Мужской,NaN,NaN,NaN,47002/B20,NaN,NaN,2636-20,Центр_Алмазова,NaN,NaN,NaN


In [359]:
# Закодируем пол
data["Gender"].value_counts()

Женский    546
Мужской    247
Name: Gender, dtype: int64

In [360]:
gender_replace = {"Gender": {"Женский": 1, "Мужской": -1}}
data = data.replace(gender_replace)
data

,PatientName,Bithday,Gender,BloodGroup,Rezus,InvGroup,NumberContract,Department0,Department1,Department2,Department3,FIOUl,TypeHospital,SendingOrganization
0,GACAAHY,19470129,1,NaN,NaN,NaN,400/A07,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
1,GACAAHe,19461001,1,NaN,NaN,NaN,406/A07,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
2,GACAAKt,19490409,-1,NaN,NaN,NaN,577/A07,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
3,GACAAVB,19650323,-1,NaN,NaN,NaN,491/A08,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
4,GACAAXQ,19510224,-1,NaN,NaN,NaN,610/A08,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,GACBAНg,19731118.0,1,NaN,NaN,NaN,90266/B18,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
789,GACBAХH,19580416.0,1,NaN,NaN,NaN,16503/B19,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
790,GACBAкЪ,20030724.0,-1,NaN,NaN,NaN,91027/B19,NaN,NaN,NaN,Центр_Алмазова,NaN,NaN,NaN
791,GACBAэы,19970805.0,-1,NaN,NaN,NaN,47002/B20,NaN,NaN,2636-20,Центр_Алмазова,NaN,NaN,NaN


In [361]:
# Вычислить возраст Age
# Алгоритм: из data берем Birthday (переводим в timestamp)
# Из dataset берем StartEpizode
# Считаем разницу, выделяем года, сохраняем

# преборазуем дату float->date()
for i in data.index:
    date = re.findall(r"(\d{4})(\d{2})(\d{2})", str(int(data["Bithday"][i])))
    date = '-'.join(date[0])
    date = datetime.strptime(date, "%Y-%m-%d").date()
    
    data["Bithday"][i] = date

/tmp/ipykernel_24556/2964224970.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Bithday"][i] = date


In [362]:
birthday_lst = data["Bithday"].tolist()
start_epizode_lst = dataset["StartEpizode"].tolist()

age_lst = []

for i in range(len(birthday_lst)):
    start_epizode_lst[i] = datetime.strptime(start_epizode_lst[i], "%Y-%m-%d").date()
    age_lst.append((start_epizode_lst[i] - birthday_lst[i]).days / 365)

In [363]:
# добавление новых столбцов в dataset_target.csv
dataset["Age"] = age_lst
dataset["Gender"] = data["Gender"].tolist()
dataset.to_csv("dataet_target")
dataset

,Name,WeightInit,Height,DateDiff,WeightDiff,Target,TargetWeight,StartEpizode,ObesityDegree,Age,Gender
0,GACAAHY,75.0,150.0,55,3.0,1,78.0,2012-09-08,1.0,65.654795,1
1,GACAAHe,110.0,158.0,156,-5.0,-1,105.0,2011-04-19,3.0,64.591781,1
2,GACAAKt,100.0,176.0,32,0.0,0,100.0,2013-05-20,1.0,64.156164,-1
3,GACAAVB,97.0,168.5,292,1.0,1,98.0,2015-01-24,1.0,49.873973,-1
4,GACAAXQ,100.0,172.5,358,0.0,0,100.0,2018-01-23,1.0,66.958904,-1
...,...,...,...,...,...,...,...,...,...,...,...
788,GACBAНg,100.0,168.0,317,2.0,1,102.0,2017-06-20,2.0,43.616438,1
789,GACBAХH,98.0,174.0,233,-6.0,-1,92.0,2019-04-24,1.0,61.063014,1
790,GACBAкЪ,116.0,182.0,326,-14.0,-1,102.0,2019-11-22,1.0,16.342466,-1
791,GACBAэы,130.0,189.0,142,4.0,1,134.0,2020-11-16,2.0,23.298630,-1


### Extract data from AllEpizodes20210707.txt (?)

In [11]:
path = 'Obesity/Analisis/AllEpizodes20210707.txt'
CHUNK_SIZE = 1000

df_iterator = pd.read_csv(
    path, 
    sep='\t',
    encoding='cp1251',
    chunksize=CHUNK_SIZE)



In [15]:
tmp_df = df_iterator.get_chunk(5)
tmp_df.columns

Index(['Epizode', 'Organization', 'IdEpozode', 'StartEpizode', 'AdmissionDate',
       'EndEpizode', 'GoalEpisode', 'TypeEpisode', 'Department',
       'PositionResource', 'DoctorName', 'EpizodeResult', 'TreatmentOutcome',
       'ICBCode', 'SendingInstitution', 'FullLastDiagnosis',
       'DescriptionClinicalDiagnosis', 'Time', 'Unnamed: 18', 'Birthday',
       'Sex'],
      dtype='object')

In [17]:
tmp_df.iloc[0, :]

Epizode                          #AAAAABAAA
Organization                         СП.АРМ
IdEpozode                               NaN
StartEpizode                       20130329
AdmissionDate                      20130329
EndEpizode                              NaN
GoalEpisode                             NaN
TypeEpisode                     АМБУЛАТОРНО
Department                              NaN
PositionResource                        NaN
DoctorName                              NaN
EpizodeResult                           NaN
TreatmentOutcome                        NaN
ICBCode                                 NaN
SendingInstitution                      NaN
FullLastDiagnosis                       NaN
DescriptionClinicalDiagnosis            NaN
Time                                    NaN
Unnamed: 18                             NaN
Birthday                            Женский
Sex                                     NaN
Name: 5, dtype: object

### Extract data from TotalAnalisis

Исследовать датасеты с анализами

dataset.csv - все пациенты с диагнозом "ожирение"

analysis_weight_df.csv - пациенты, у которых есть запись в колонке "Вес"

analisis_ids_df.csv - записи пациентов с диагнозом "ожирение"

In [3]:
# dataset for all patients with obesity diagnosis
data = pd.read_csv("dataset.csv", index_col=0)
patient_ids = data["Name"].unique()
print(f"Number of names: {len(patient_ids)}")

Number of names: 8092


In [ ]:
path = 'Data/TotalAnalisis_20210707.txt'

CHUNK_SIZE = 2000

df_iterator = pd.read_csv(
    path, 
    sep='\t',
    encoding='cp1251',
    chunksize=CHUNK_SIZE)

In [18]:
path = "/home/alexandra/ITMO/MedStat/Data for all nosologes 070721/"
file = "ColumnsTotalAnalisisAllPatientsNewBase_eng.txt"
file2 = "ColumnsTotalAnalisisAllPatientsNewBase.txt"
analisis_name_en = pd.read_csv(path+file, sep='\t', encoding='cp1251')
analisis_name_ru = pd.read_csv(path+file2, sep='\t', encoding='cp1251')

In [19]:
for i in analisis_name_ru.columns:
    print(i)

epizod
Name
Тип_обработки_данных
Протромбиновое время
Протромбин (по Квику)
Протеин S в крови
Протеин C в крови
Мочевина
Билирубин общий
Билирубин прямой
Билирубин непрямой
Триглицериды
Креатинин
Холестерин общий
Мочевая кислота
Кальций общий
Железо
Глюкоза (сыв.)
Альбумин
Калий
Кальций ионизированный
АСТ
АЛТ
Натрий
Общий белок
Unnamed: 25
Креатинин ДиаС
Б0045.1
Холестерин ЛПВП
Б0050.1
Коэф. атерогенности
Холестерин ЛПОНП
Биоматериал
С-реактивный белок (СРБ) колич.
Альбумин (электофорез)
Гликозилированный гемоглобин
Скорость клубочковой фильтрации
Глюкоза после нагрузки
Соотношение Альбумин/Креатинин
Холестерин ЛПНП
Магний
Рост
Вес
Площадь поверхности тела
WBC- Лейкоциты
Ретикулоциты #
HGB- Гемоглобин
Моноциты, эозинофилы, базофилы %
HCT- Гематокрит
Моноциты, эозинофилы, базофилы #
Лейкоциты
Билирубин общий.1
Билирубин неонатальный
Билирубин прямой.1
Билирубин непрямой.1
Общий белок.1
Мочевина.1
Натрий.1
Калий.1
.1
Триглицериды.1
Хлориды
С-реактивный белок
Фибриноген
Кальций общий.1
Ма

In [ ]:
analisis_data = []
for chunk in tqdm(df_iterator):
    for i in range(len(chunk)):
        if (chunk.iloc[i]["Name"] in patient_ids):
            analisis_data.append(chunk.iloc[i])

In [ ]:
analisis_df = pd.DataFrame(analisis_data)
analisis_df.to_csv("analisis_ids_df.csv")

# Rename columns
analisis_ids = pd.read_csv("analisis_ids_df.csv", index_col=0)
analisis_ids.columns = tmp_col
analisis_ids.to_csv("analisis_ids_df.csv")

In [273]:
data_analisis = pd.read_csv("analisis_ids_df.csv", index_col=0)
# extract only patients from dataset with target

data = pd.read_csv("dataset_target.csv", index_col=0)
patient_ids = data["Name"].unique()
analisis_lst = []

for patient in tqdm(patient_ids):
    tmp_df = data_analisis[data_analisis["Name"]==patient]
    tmp_lst = tmp_df.values.tolist()
    for i in range(len(tmp_lst)):
        analisis_lst.append(tmp_lst[i])

/tmp/ipykernel_24556/3918595823.py:1: DtypeWarning: Columns (3,6,7,10,11,16,17,21,27,28,35,36,38,39,40,41,42,43,44,48,50,52,53,54,55,57,58,59,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,81,82,83,85,86,87,89,90,92,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,117,118,120,121,122,123,124,125,126,127,128,134,135,136,137,139,140,141,142,143,144,145,146,147,150,151,152,153,154,156,157,159,160,161,163,164,165,166,167,168,169,170,171,172) have mixed types. Specify dtype option on import or set low_memory=False.
  data_analisis = pd.read_csv("analisis_ids_df.csv", index_col=0)
100%|█████████████████████████████████████████| 793/793 [00:09<00:00, 80.72it/s]


In [274]:
col = data_analisis.columns
data_analisis = pd.DataFrame(analisis_lst, columns=col)

In [275]:
data_analisis

,epizod,Name,Data_processing_type,Prothrombin time,Prothrombin (according to Quick),Protein S in the blood,Protein C in the blood,Urea,Bilirubin total,Bilirubin straight,...,Troponin,Troponin I,Calcium total. 4,Thyroid-stimulating hormone,Total cholesterol. 1,HDL cholesterol. 3,Triglycerides. 4,VLDL cholesterol. 2,LDL cholesterol (calculated),E0083epizod
0,GACAAKtAAGAAB12:48AAAA,GACAAKt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GACAAKtAAGAAD11:08AAAA,GACAAKt,NaN,NaN,NaN,NaN,NaN,"6.50 63004,40139 КМЛ MIF||ARCH",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GACAAKtAAGAAD11:08BAAA,GACAAKt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GACAAKtAAGAAD11:09AAAA,GACAAKt,NaN,NaN,NaN,NaN,NaN,NaN,"8.90 63004,40140 КМЛ MIF||ARCH",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GACAAKtAAGAAD11:09BAAA,GACAAKt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48753,GACBB6FAAaAAA06:33ABAA,GACBB6F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48754,GACBB6FAAbAAA08:20ABAA,GACBB6F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48755,GACBB6FAAcAAA08:20BBAA,GACBB6F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48756,GACBB6FAAcAAA08:20CBAA,GACBB6F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
tmp_lst = data_analisis.iloc[2,:]

Алгоритм извлечения анализов:
- извлекаем df по имени 
- формируем ненулевые списки по столбцам
- извлекаем первое вхождение 
- сохраняем в список пациента

In [276]:
# протестируем алгоритм для одном пациенте
tmp_df = data_analisis[data_analisis["Name"]=="GACAAKt"]
cols = tmp_df.columns[range(1, len(tmp_df.columns))]
tmp_df = tmp_df[cols]
row_lst = ["GACAAKt"]

for col in cols[range(1, len(cols))]:
    tmp_lst = [x for x in tmp_df[col] if pd.isnull(x) == False and x != 'nan']
    
    
    if len(tmp_lst) == 0:
        row_lst.append(np.nan)
    else:
        row_lst.append(tmp_lst[0])

print(row_lst)


['GACAAKt', nan, ' 12.7 63672,40178  СНЯ MIF||STACOM2       ', ' 97 63672,40178  СНЯ MIF||STACOM2       ', nan, nan, ' 6.50  63004,40139  КМЛ MIF||ARCH ', ' 8.90  63004,40140  КМЛ MIF||ARCH ', nan, nan, ' 1.62 63894,45789  Shidakova_NA MIF||ARCH    EXPC    ', ' 112  63004,40139  КМЛ MIF||ARCH ', ' 7.40 63672,39311  ЮВА MIF||ARCH    HIGH    ', nan, ' 2.26  63004,40140  КМЛ MIF||ARCH ', nan, ' 5.69  63004,40140  КМЛ MIF||ARCH ', ' 41.00  63004,40140  КМЛ MIF||ARCH ', ' 5.0  63004,40140  КМЛ MIF||ARCH ', nan, ' 36.0  63004,40140  КМЛ MIF||ARCH ', ' 51.0  63004,40140  КМЛ MIF||ARCH ', ' 141  63004,40140  КМЛ MIF||ARCH ', ' 72.00  63004,40140  КМЛ MIF||ARCH ', nan, nan, nan, ' 0.91 63894,45789  Shidakova_NA MIF||ARCH    LOW    ', ' 0.91 63894,45789  Shidakova_NA MIF||ARCH    LOW    ', ' 3.23 63894,45789  Shidakova_NA        ', ' 0.74 63894,45789  Shidakova_NA        ', ' Сыворотка крови 65794,48803  Baalina_EV          N', ' 4.48 63784,43152  Shidakova_NA MIF||ARCH       ', nan, nan, ' 49.7

In [277]:
names_lst = data_analisis["Name"].unique()
cols = data_analisis.columns
analisis_lst = []

for name in tqdm(names_lst):
    tmp_df = data_analisis[data_analisis["Name"]==name]
    cols = tmp_df.columns[range(1, len(tmp_df.columns))]
    tmp_df = tmp_df[cols]
    row_lst = [name]

    for col in cols[range(1, len(cols))]:
        tmp_lst = [x for x in tmp_df[col] if pd.isnull(x) == False and x != 'nan']
    
    
        if len(tmp_lst) == 0:
            row_lst.append(0)
        else:
            row_lst.append(tmp_lst[0])

    analisis_lst.append(row_lst)



100%|█████████████████████████████████████████| 565/565 [00:08<00:00, 67.10it/s]


In [278]:
analisis_df = pd.DataFrame(analisis_lst, columns=cols)

In [279]:
analisis_df

,Name,Data_processing_type,Prothrombin time,Prothrombin (according to Quick),Protein S in the blood,Protein C in the blood,Urea,Bilirubin total,Bilirubin straight,Bilirubin indirect,...,Troponin,Troponin I,Calcium total. 4,Thyroid-stimulating hormone,Total cholesterol. 1,HDL cholesterol. 3,Triglycerides. 4,VLDL cholesterol. 2,LDL cholesterol (calculated),E0083epizod
0,GACAAKt,0,"12.7 63672,40178 СНЯ MIF||STACOM2","97 63672,40178 СНЯ MIF||STACOM2",0,0,"6.50 63004,40139 КМЛ MIF||ARCH","8.90 63004,40140 КМЛ MIF||ARCH",0,0,...,0,"0.035 63008,39239 КМЛ MIF||ARCH2",0,0,0,0,0,0,0,0
1,GACAAXQ,0,"21.2 64632,37960 Bornevskaya_PG MIF||STACOM2...","45 64632,37961 Bornevskaya_PG MIF||STACOM2 ...",0,0,"9.60 65029,48963 Rassokhina_LG MIF||ARCH ...","7.00 65029,48807 Rassokhina_LG MIF||ARCH ...",0,0,...,0,"0.0100 64633,68200 Garnega_TS MIF||ACCESS ...",0,0,0,0,0,0,0,0
2,GACAAZy,0,0,0,0,0,0,0,0,0,...,0,0.000 Ком. Результат теста находится в рефере...,0,0,0,0,0,0,0,0
3,GACAAil,0,"11.8 62962,44050 ЮВА MIF||STACOM2","124 62962,44050 ЮВА MIF||STACOM2",0,0,0,"8.50 62962,43705 ЮВА MIF||ARCH",0,0,...,0,0,0,0,0,0,0,0,0,0
4,GACABVe,0,"13.562312,38289БОНMIF||STACOM2","9862312,38289БОНMIF||STACOM2",0,0,4.50,12.90,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,GACBAНg,0,"13.7 64772,37019 Smozhenkova_EV MIF||STACOM2...","91 64772,37019 Smozhenkova_EV MIF||STACOM2 ...","63.00 64784,44191 Smozhenkova_EV MIF||STACOM...","128.00 64784,44191 Smozhenkova_EV MIF||STACO...","4.20 65038,49496 Shidakova_NA MIF||ARCH E...","9.30 65038,49493 Shidakova_NA MIF||ARCH ...",0,0,...,0,0,0,0,0,0,0,0,0,0
561,GACBAХH,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
562,GACBAкЪ,0,0,0,0,0,"4.60 65356,51238 Baalina_EV MIF||ARCH CNT...","13.70 65356,51238 Baalina_EV MIF||ARCH ...",0,0,...,0,0,0,0,0,0,0,0,0,0
563,GACBAэы,0,0,0,0,0,"4.80 65841,55364 Baalina_EV MIF||ARCH N",0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [280]:
# remove columns with NaNs
# for one column
lst = analisis_df["Bilirubin indirect"]
ids = []
for i in range(len(lst)):
    if lst[i] == 0:
        ids.append(i)

delete_elements(lst, ids)
if len(lst) == 0:
    analisis_df = analisis_df.drop("Bilirubin indirect", axis=1)

print(analisis_df)

        Name  Data_processing_type  \
0    GACAAKt                     0   
1    GACAAXQ                     0   
2    GACAAZy                     0   
3    GACAAil                     0   
4    GACABVe                     0   
..       ...                   ...   
560  GACBAНg                     0   
561  GACBAХH                     0   
562  GACBAкЪ                     0   
563  GACBAэы                     0   
564  GACBB6F                     0   

                                      Prothrombin time  \
0            12.7 63672,40178  СНЯ MIF||STACOM2          
1     21.2 64632,37960  Bornevskaya_PG MIF||STACOM2...   
2                                                    0   
3                 11.8  62962,44050  ЮВА MIF||STACOM2    
4                       13.562312,38289БОНMIF||STACOM2   
..                                                 ...   
560   13.7 64772,37019  Smozhenkova_EV MIF||STACOM2...   
561                                                  0   
562                  

In [281]:
# for each column
for col in analisis_df.columns:
    lst = analisis_df[col]
    ids = []
    for i in range(len(lst)):
        if lst[i] == 0:
            ids.append(i)

    delete_elements(lst, ids)
    if len(lst) == 0:
        analisis_df = analisis_df.drop(col, axis=1)    

In [282]:
analisis_df  

,Name,Prothrombin time,Prothrombin (according to Quick),Protein S in the blood,Protein C in the blood,Urea,Bilirubin total,Bilirubin straight,Bilirubin indirect,Triglycerides,...,Troponin,Troponin I,Calcium total. 4,Thyroid-stimulating hormone,Total cholesterol. 1,HDL cholesterol. 3,Triglycerides. 4,VLDL cholesterol. 2,LDL cholesterol (calculated),E0083epizod
0,GACAAKt,"12.7 63672,40178 СНЯ MIF||STACOM2","97 63672,40178 СНЯ MIF||STACOM2",0,0,"6.50 63004,40139 КМЛ MIF||ARCH","8.90 63004,40140 КМЛ MIF||ARCH",0,0,"1.62 63894,45789 Shidakova_NA MIF||ARCH E...",...,0,"0.035 63008,39239 КМЛ MIF||ARCH2",0,0,0,0,0,0,0,0
1,GACAAXQ,"21.2 64632,37960 Bornevskaya_PG MIF||STACOM2...","45 64632,37961 Bornevskaya_PG MIF||STACOM2 ...",0,0,"9.60 65029,48963 Rassokhina_LG MIF||ARCH ...","7.00 65029,48807 Rassokhina_LG MIF||ARCH ...",0,0,"1.04 64630,49583 Shidakova_NA MIF||ARCH",...,0,"0.0100 64633,68200 Garnega_TS MIF||ACCESS ...",0,0,0,0,0,0,0,0
2,GACAAZy,0,0,0,0,0,0,0,0,0,...,0,0.000 Ком. Результат теста находится в рефере...,0,0,0,0,0,0,0,0
3,GACAAil,"11.8 62962,44050 ЮВА MIF||STACOM2","124 62962,44050 ЮВА MIF||STACOM2",0,0,0,"8.50 62962,43705 ЮВА MIF||ARCH",0,0,"2.31 63067,38661 ИМЛ MIF||ARCH",...,0,0,0,0,0,0,0,0,0,0
4,GACABVe,"13.562312,38289БОНMIF||STACOM2","9862312,38289БОНMIF||STACOM2",0,0,4.50,12.90,0,0,1.95,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,GACBAНg,"13.7 64772,37019 Smozhenkova_EV MIF||STACOM2...","91 64772,37019 Smozhenkova_EV MIF||STACOM2 ...","63.00 64784,44191 Smozhenkova_EV MIF||STACOM...","128.00 64784,44191 Smozhenkova_EV MIF||STACO...","4.20 65038,49496 Shidakova_NA MIF||ARCH E...","9.30 65038,49493 Shidakova_NA MIF||ARCH ...",0,0,"2.84 65038,49493 Shidakova_NA MIF||ARCH H...",...,0,0,0,0,0,0,0,0,0,0
561,GACBAХH,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
562,GACBAкЪ,0,0,0,0,"4.60 65356,51238 Baalina_EV MIF||ARCH CNT...","13.70 65356,51238 Baalina_EV MIF||ARCH ...",0,0,"2.12 65356,51238 Baalina_EV MIF||ARCH HIG...",...,0,0,0,0,0,0,0,0,0,0
563,GACBAэы,0,0,0,0,"4.80 65841,55364 Baalina_EV MIF||ARCH N",0,0,0,"1.75 65841,55365 Baalina_EV MIF||ARCH HIG...",...,0,0,0,0,0,0,0,0,0,0


In [283]:
for col in analisis_df.columns:
    n = 0
    lst = analisis_df[col]
    for elem in lst:
        if elem == 0:
            n += 1
    if n > 300:
        analisis_df = analisis_df.drop(col, axis=1)

In [284]:
analisis_df

,Name,Prothrombin time,Prothrombin (according to Quick),Bilirubin total,Creatinine,Total cholesterol,Glucose (syv.),Potassium,AST,ALT,WBC- Leukocytes,HGB- Hemoglobin,HCT-Hematocrit
0,GACAAKt,"12.7 63672,40178 СНЯ MIF||STACOM2","97 63672,40178 СНЯ MIF||STACOM2","8.90 63004,40140 КМЛ MIF||ARCH","112 63004,40139 КМЛ MIF||ARCH","7.40 63672,39311 ЮВА MIF||ARCH HIGH","5.69 63004,40140 КМЛ MIF||ARCH","5.0 63004,40140 КМЛ MIF||ARCH","36.0 63004,40140 КМЛ MIF||ARCH","51.0 63004,40140 КМЛ MIF||ARCH","3.9 63002,46106 ФЮВ MIF||SYSM21","140.0 63002,46106 ФЮВ MIF||SYSM21","41.7 63002,46106 ФЮВ MIF||SYSM21"
1,GACAAXQ,"21.2 64632,37960 Bornevskaya_PG MIF||STACOM2...","45 64632,37961 Bornevskaya_PG MIF||STACOM2 ...","7.00 65029,48807 Rassokhina_LG MIF||ARCH ...","123 64630,45467 Bornevskaya_PG MIF||ARCH_1 ...","3.12 64630,49583 Shidakova_NA MIF||ARCH E...","9.30 64636,24782 Maveeva_YuA MIF||ARCH_1 ...","5.70 64630,45467 Bornevskaya_PG MIF||ARCH_1 ...","21.0 64630,45467 Bornevskaya_PG MIF||ARCH_1 ...","37.0 64630,45467 Bornevskaya_PG MIF||ARCH_1 ...","8.2 64630,44010 Bornevskaya_PG MIF||XT1800_1...","155.0 64630,44010 Bornevskaya_PG MIF||XT1800...","43.4 64630,44010 Bornevskaya_PG MIF||XT1800_..."
2,GACAAZy,0,0,0,0,0,0,0,0,0,0,0,0
3,GACAAil,"11.8 62962,44050 ЮВА MIF||STACOM2","124 62962,44050 ЮВА MIF||STACOM2","8.50 62962,43705 ЮВА MIF||ARCH","72 62962,43705 ЮВА MIF||ARCH","3.63 62962,43705 ЮВА MIF||ARCH","6.12 62962,43705 ЮВА MIF||ARCH","4.20 62962,43705 ЮВА MIF||ARCH","19.0 62962,43705 ЮВА MIF||ARCH","18.0 62962,43705 ЮВА MIF||ARCH","6.5 62962,47825 МЮВ MIF||ACTD2","126.0 62962,47825 МЮВ MIF||ACTD2","39.1 62962,47825 МЮВ MIF||ACTD2"
4,GACABVe,"13.562312,38289БОНMIF||STACOM2","9862312,38289БОНMIF||STACOM2",12.90,62,4.48,6.18,4.30,14.0,11.0,"5.762312,35293ШОВMIF||COULT","130.062312,35294ШОВMIF||COULT","36.562312,35294ШОВMIF||COULT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,GACBAНg,"13.7 64772,37019 Smozhenkova_EV MIF||STACOM2...","91 64772,37019 Smozhenkova_EV MIF||STACOM2 ...","9.30 65038,49493 Shidakova_NA MIF||ARCH ...","60 65038,49492 Shidakova_NA MIF||ARCH EXP...","3.71 65038,49493 Shidakova_NA MIF||ARCH E...","5.76 65038,49493 Shidakova_NA MIF||ARCH H...","4.70 65038,49493 Shidakova_NA MIF||ARCH E...","17.0 65038,49493 Shidakova_NA MIF||ARCH ...","17.0 65038,49493 Shidakova_NA MIF||ARCH E...","8.8 65038,50221 Feisov_ES MIF||CELLDY_R F...","136.8 65038,50221 Feisov_ES MIF||CELLDY_R ...","43.7 65038,50221 Feisov_ES MIF||CELLDY_R ..."
561,GACBAХH,0,0,0,0,0,0,0,0,0,"10.0 65128,36417 Baranova_TN MIF||XT1800 ...","149.0 65128,36417 Baranova_TN MIF||XT1800 ...","44.5 65128,36417 Baranova_TN MIF||XT1800 ..."
562,GACBAкЪ,0,0,"13.70 65356,51238 Baalina_EV MIF||ARCH ...","72 65357,40999 Shidakova_NA MIF||ARCH EXP...","5.79 65356,51238 Baalina_EV MIF||ARCH HIG...","4.20 65356,51238 Baalina_EV MIF||ARCH EXP...","4.40 65356,50685 Shidakova_NA MIF||ARCH E...","20.0 65356,51238 Baalina_EV MIF||ARCH N","25.0 65356,51238 Baalina_EV MIF||ARCH N","7.6 65356,45575 Roy_AS MIF||XS1000I N N","146.0 65356,45575 Roy_AS MIF||XS1000I N ...","42.1 65356,45575 Roy_AS MIF||XS1000I N ..."
563,GACBAэы,0,0,0,"78.60 65841,55428 Bornevskaya_PG MIF||ARCH_1...","3.79 65841,55461 Baalina_EV MIF||ARCH EXP...","4.61 65841,55431 Bornevskaya_PG MIF||ARCH_1 ...","4.00 65841,55430 Bornevskaya_PG MIF||ARCH_1 ...","23.0 65841,55429 Bornevskaya_PG MIF||ARCH_1 ...","42.0 65841,55429 Bornevskaya_PG MIF||ARCH_1 ...","12.6 65841,54322 Blokhina_NV MIF||XT1800_1 ...","155.0 65841,54321 Blokhina_NV MIF||XT1800_1 ...","44.4 65841,54321 Blokhina_NV MIF||XT1800_1 ..."


Регулярные выражения для столбцов:
- \d{2}\.\d{1} "Prothrombin time", "AST", "ALT", "HCT-Hematocrit"
- сначала \d{2}\s если nan, то \d{3} "Prothrombin (according to Quick)", "Creatinine"
- сначала \d{1}\.\d{2} если nan, то \d{2}\.\d{2} "Bilirubin total", "Total Cholesterol", "Glucose (syv.)"
- \d{1}\.\d{1} если nan, то \d{2}\.\d{1} "Potassium", "WBC- Leukocytes"
- \d{3}\.\d{1} "HGB- Hemoglobin"

In [285]:
# выделим показания анализов
# для одного столбца

records = analisis_df["Prothrombin time"].tolist()

analisis_lst = []
tmp_ind = analisis_df.columns.tolist().index("Prothrombin time")

for i in range(len(analisis_df)):
    record = analisis_df.iloc[i, tmp_ind]
    if record == 0:
        analisis_lst.append(0)
        continue
        
    analysis = re.findall(r"\d{2}\.\d{1}", record)
    
    if len(analysis) == 0:
        analisis_lst.append(0)
    else:
        analisis_lst.append(float(analysis[0]))

In [286]:
analisis_df_copy = analisis_df.copy()
analisis_df_copy

,Name,Prothrombin time,Prothrombin (according to Quick),Bilirubin total,Creatinine,Total cholesterol,Glucose (syv.),Potassium,AST,ALT,WBC- Leukocytes,HGB- Hemoglobin,HCT-Hematocrit
0,GACAAKt,"12.7 63672,40178 СНЯ MIF||STACOM2","97 63672,40178 СНЯ MIF||STACOM2","8.90 63004,40140 КМЛ MIF||ARCH","112 63004,40139 КМЛ MIF||ARCH","7.40 63672,39311 ЮВА MIF||ARCH HIGH","5.69 63004,40140 КМЛ MIF||ARCH","5.0 63004,40140 КМЛ MIF||ARCH","36.0 63004,40140 КМЛ MIF||ARCH","51.0 63004,40140 КМЛ MIF||ARCH","3.9 63002,46106 ФЮВ MIF||SYSM21","140.0 63002,46106 ФЮВ MIF||SYSM21","41.7 63002,46106 ФЮВ MIF||SYSM21"
1,GACAAXQ,"21.2 64632,37960 Bornevskaya_PG MIF||STACOM2...","45 64632,37961 Bornevskaya_PG MIF||STACOM2 ...","7.00 65029,48807 Rassokhina_LG MIF||ARCH ...","123 64630,45467 Bornevskaya_PG MIF||ARCH_1 ...","3.12 64630,49583 Shidakova_NA MIF||ARCH E...","9.30 64636,24782 Maveeva_YuA MIF||ARCH_1 ...","5.70 64630,45467 Bornevskaya_PG MIF||ARCH_1 ...","21.0 64630,45467 Bornevskaya_PG MIF||ARCH_1 ...","37.0 64630,45467 Bornevskaya_PG MIF||ARCH_1 ...","8.2 64630,44010 Bornevskaya_PG MIF||XT1800_1...","155.0 64630,44010 Bornevskaya_PG MIF||XT1800...","43.4 64630,44010 Bornevskaya_PG MIF||XT1800_..."
2,GACAAZy,0,0,0,0,0,0,0,0,0,0,0,0
3,GACAAil,"11.8 62962,44050 ЮВА MIF||STACOM2","124 62962,44050 ЮВА MIF||STACOM2","8.50 62962,43705 ЮВА MIF||ARCH","72 62962,43705 ЮВА MIF||ARCH","3.63 62962,43705 ЮВА MIF||ARCH","6.12 62962,43705 ЮВА MIF||ARCH","4.20 62962,43705 ЮВА MIF||ARCH","19.0 62962,43705 ЮВА MIF||ARCH","18.0 62962,43705 ЮВА MIF||ARCH","6.5 62962,47825 МЮВ MIF||ACTD2","126.0 62962,47825 МЮВ MIF||ACTD2","39.1 62962,47825 МЮВ MIF||ACTD2"
4,GACABVe,"13.562312,38289БОНMIF||STACOM2","9862312,38289БОНMIF||STACOM2",12.90,62,4.48,6.18,4.30,14.0,11.0,"5.762312,35293ШОВMIF||COULT","130.062312,35294ШОВMIF||COULT","36.562312,35294ШОВMIF||COULT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,GACBAНg,"13.7 64772,37019 Smozhenkova_EV MIF||STACOM2...","91 64772,37019 Smozhenkova_EV MIF||STACOM2 ...","9.30 65038,49493 Shidakova_NA MIF||ARCH ...","60 65038,49492 Shidakova_NA MIF||ARCH EXP...","3.71 65038,49493 Shidakova_NA MIF||ARCH E...","5.76 65038,49493 Shidakova_NA MIF||ARCH H...","4.70 65038,49493 Shidakova_NA MIF||ARCH E...","17.0 65038,49493 Shidakova_NA MIF||ARCH ...","17.0 65038,49493 Shidakova_NA MIF||ARCH E...","8.8 65038,50221 Feisov_ES MIF||CELLDY_R F...","136.8 65038,50221 Feisov_ES MIF||CELLDY_R ...","43.7 65038,50221 Feisov_ES MIF||CELLDY_R ..."
561,GACBAХH,0,0,0,0,0,0,0,0,0,"10.0 65128,36417 Baranova_TN MIF||XT1800 ...","149.0 65128,36417 Baranova_TN MIF||XT1800 ...","44.5 65128,36417 Baranova_TN MIF||XT1800 ..."
562,GACBAкЪ,0,0,"13.70 65356,51238 Baalina_EV MIF||ARCH ...","72 65357,40999 Shidakova_NA MIF||ARCH EXP...","5.79 65356,51238 Baalina_EV MIF||ARCH HIG...","4.20 65356,51238 Baalina_EV MIF||ARCH EXP...","4.40 65356,50685 Shidakova_NA MIF||ARCH E...","20.0 65356,51238 Baalina_EV MIF||ARCH N","25.0 65356,51238 Baalina_EV MIF||ARCH N","7.6 65356,45575 Roy_AS MIF||XS1000I N N","146.0 65356,45575 Roy_AS MIF||XS1000I N ...","42.1 65356,45575 Roy_AS MIF||XS1000I N ..."
563,GACBAэы,0,0,0,"78.60 65841,55428 Bornevskaya_PG MIF||ARCH_1...","3.79 65841,55461 Baalina_EV MIF||ARCH EXP...","4.61 65841,55431 Bornevskaya_PG MIF||ARCH_1 ...","4.00 65841,55430 Bornevskaya_PG MIF||ARCH_1 ...","23.0 65841,55429 Bornevskaya_PG MIF||ARCH_1 ...","42.0 65841,55429 Bornevskaya_PG MIF||ARCH_1 ...","12.6 65841,54322 Blokhina_NV MIF||XT1800_1 ...","155.0 65841,54321 Blokhina_NV MIF||XT1800_1 ...","44.4 65841,54321 Blokhina_NV MIF||XT1800_1 ..."


#### Основные анализы их диапазоны

Prothrombin time - a prothrombin time (PT) test measures how long it takes for a clot to form in a blood sample
norm: 11—16 секунд

Prothrombin (according to Quick)

In [287]:
# для всех записей

cols = analisis_df.columns
analisis_df_copy = analisis_df.copy()

for col in cols:
    if col == "Name": 
        continue
    
    elif col in ["Prothrombin (according to Quick)", "Creatinine"]:
        records = analisis_df_copy[col].tolist()

        analisis_lst = []
        tmp_ind = analisis_df_copy.columns.tolist().index(col)

        for i in range(len(analisis_df_copy)):
            record = analisis_df_copy.iloc[i, tmp_ind]
            if record == 0:
                analisis_lst.append(np.nan)
                continue
        
            analysis = re.findall(r"^\s*\d{3}\s*", record)
            
            if len(analysis) == 0:
                analysis = re.findall("^\s*\d{2}\s*", record)
            if len(analysis) == 0:
                analisis_lst.append(np.nan)
            else:
                analisis_lst.append(float(analysis[0]))
    
    elif col in ["Potassium", "WBC- Leukocytes", "Prothrombin time", "AST", "ALT", 
                 "HCT-Hematocrit", "Total cholesterol", "Bilirubin total", "Glucose (syv.)"]:
        records = analisis_df_copy[col].tolist()

        analisis_lst = []
        tmp_ind = analisis_df_copy.columns.tolist().index(col)

        for i in range(len(analisis_df_copy)):
            record = analisis_df_copy.iloc[i, tmp_ind]
            if record == 0:
                analisis_lst.append(np.nan)
                continue
        
            analysis = re.findall(r"^\s*\d{1}\.\d{1}", record)
    
            if len(analysis) == 0:
                analysis = re.findall(r"^\s*\d{2}\.\d{1}", record)
            if len(analysis) == 0:
                analisis_lst.append(np.nan)
            else:
                analisis_lst.append(float(analysis[0]))   
    
    elif col in ["HGB- Hemoglobin"]:
        records = analisis_df_copy[col].tolist()

        analisis_lst = []
        tmp_ind = analisis_df_copy.columns.tolist().index(col)

        for i in range(len(analisis_df_copy)):
            record = analisis_df_copy.iloc[i, tmp_ind]
            if record == 0:
                analisis_lst.append(np.nan)
                continue
        
            analysis = re.findall(r"^\s*\d{3}\.\d{1}", record)
    
            if len(analysis) == 0:
                analisis_lst.append(np.nan)
            else:
                analisis_lst.append(float(analysis[0]))
    
    analisis_df_copy[col] = analisis_lst

In [267]:
tmp_1 = analisis_df["HGB- Hemoglobin"]
tmp_2 = analisis_df_copy["HGB- Hemoglobin"]

for i in range(len(tmp_1)):
    print(tmp_1[i], tmp_2[i])

 140.0  63002,46106  ФЮВ MIF||SYSM21  140.0
 155.0 64630,44010  Bornevskaya_PG MIF||XT1800_1    N      155.0
0 0.0
 126.0  62962,47825  МЮВ MIF||ACTD2  126.0
 130.062312,35294ШОВMIF||COULT 130.0
0 0.0
 143.0 143.0
 155.0 155.0
 129.0  63035,35833  ДОБ MIF||XT1800  129.0
 156.0 65337,38745  Goncharova_IM MIF||MICROS    H      N 156.0
 136.0  62742,51155  ШОВ MIF||COULT 136.0
 132.0  62894,45668  ГМВ MIF||COULT  132.0
 132.062307,39457РЕВMIF||COULT 132.0
 134.0 65577,46428  ДОБ MIF||XT1800    N      N 134.0
 147.8 63537,42858  ТЕВ MIF||CELLDY_R    F     147.8
 132.0 63293,36897  ТВА MIF||SYSM22        132.0
 139.0 65108,50419  Baranova_TN MIF||XT1800    N      N 139.0
 165.0  63029,44253  ДОБ MIF||XT1800  165.0
 133.0  62641,47762  ШОВ MIF||COULT 133.0
0 0.0
 160.5 64013,37963  ШОВ MIF||CELLDY_R    F     160.5
 137.0  62641,41356  ГМВ MIF||COULT 137.0
 127.062376,54129НОСMIF||COULT 127.0
 131.0 63520,34597  Pelevina_EV MIF||SYSM22        131.0
 142.0  62643,41527  ШОВ MIF||COULT 142.0
 1

In [288]:
analisis_df_copy

,Name,Prothrombin time,Prothrombin (according to Quick),Bilirubin total,Creatinine,Total cholesterol,Glucose (syv.),Potassium,AST,ALT,WBC- Leukocytes,HGB- Hemoglobin,HCT-Hematocrit
0,GACAAKt,12.7,97.0,8.9,112.0,7.4,5.6,5.0,36.0,51.0,3.9,140.0,41.7
1,GACAAXQ,21.2,45.0,7.0,123.0,3.1,9.3,5.7,21.0,37.0,8.2,155.0,43.4
2,GACAAZy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GACAAil,11.8,124.0,8.5,72.0,3.6,6.1,4.2,19.0,18.0,6.5,126.0,39.1
4,GACABVe,13.5,986.0,12.9,62.0,4.4,6.1,4.3,14.0,11.0,5.7,130.0,36.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,GACBAНg,13.7,91.0,9.3,60.0,3.7,5.7,4.7,17.0,17.0,8.8,136.8,43.7
561,GACBAХH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,149.0,44.5
562,GACBAкЪ,NaN,NaN,13.7,72.0,5.7,4.2,4.4,20.0,25.0,7.6,146.0,42.1
563,GACBAэы,NaN,NaN,NaN,78.0,3.7,4.6,4.0,23.0,42.0,12.6,155.0,44.4


In [289]:
analisis_df = analisis_df_copy

In [290]:
analisis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Name                              565 non-null    object 
 1   Prothrombin time                  315 non-null    float64
 2   Prothrombin (according to Quick)  314 non-null    float64
 3   Bilirubin total                   305 non-null    float64
 4   Creatinine                        314 non-null    float64
 5   Total cholesterol                 297 non-null    float64
 6   Glucose (syv.)                    320 non-null    float64
 7   Potassium                         298 non-null    float64
 8   AST                               321 non-null    float64
 9   ALT                               322 non-null    float64
 10  WBC- Leukocytes                   526 non-null    float64
 11  HGB- Hemoglobin                   516 non-null    float64
 12  HCT-Hema

Объединим данный показатели с основным датасетом

In [372]:
dataset = pd.read_csv("dataet_target", index_col=0)
dataset

,Name,WeightInit,Height,DateDiff,WeightDiff,Target,TargetWeight,StartEpizode,ObesityDegree,Age,Gender
0,GACAAHY,75.0,150.0,55,3.0,1,78.0,2012-09-08,1.0,65.654795,1
1,GACAAHe,110.0,158.0,156,-5.0,-1,105.0,2011-04-19,3.0,64.591781,1
2,GACAAKt,100.0,176.0,32,0.0,0,100.0,2013-05-20,1.0,64.156164,-1
3,GACAAVB,97.0,168.5,292,1.0,1,98.0,2015-01-24,1.0,49.873973,-1
4,GACAAXQ,100.0,172.5,358,0.0,0,100.0,2018-01-23,1.0,66.958904,-1
...,...,...,...,...,...,...,...,...,...,...,...
788,GACBAНg,100.0,168.0,317,2.0,1,102.0,2017-06-20,2.0,43.616438,1
789,GACBAХH,98.0,174.0,233,-6.0,-1,92.0,2019-04-24,1.0,61.063014,1
790,GACBAкЪ,116.0,182.0,326,-14.0,-1,102.0,2019-11-22,1.0,16.342466,-1
791,GACBAэы,130.0,189.0,142,4.0,1,134.0,2020-11-16,2.0,23.298630,-1


In [373]:
result = pd.merge(dataset, analisis_df, on='Name')

In [374]:
result

,Name,WeightInit,Height,DateDiff,WeightDiff,Target,TargetWeight,StartEpizode,ObesityDegree,Age,...,Bilirubin total,Creatinine,Total cholesterol,Glucose (syv.),Potassium,AST,ALT,WBC- Leukocytes,HGB- Hemoglobin,HCT-Hematocrit
0,GACAAKt,100.0,176.0,32,0.0,0,100.0,2013-05-20,1.0,64.156164,...,8.9,112.0,7.4,5.6,5.0,36.0,51.0,3.9,140.0,41.7
1,GACAAXQ,100.0,172.5,358,0.0,0,100.0,2018-01-23,1.0,66.958904,...,7.0,123.0,3.1,9.3,5.7,21.0,37.0,8.2,155.0,43.4
2,GACAAZy,86.0,162.0,307,0.0,0,86.0,2009-09-04,1.0,63.602740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GACAAil,83.0,163.0,162,-3.0,-1,80.0,2013-03-21,1.0,73.065753,...,8.5,72.0,3.6,6.1,4.2,19.0,18.0,6.5,126.0,39.1
4,GACABVe,90.0,164.0,219,-1.0,-1,89.0,2009-09-16,1.0,73.452055,...,12.9,62.0,4.4,6.1,4.3,14.0,11.0,5.7,130.0,36.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,GACBAНg,100.0,168.0,317,2.0,1,102.0,2017-06-20,2.0,43.616438,...,9.3,60.0,3.7,5.7,4.7,17.0,17.0,8.8,136.8,43.7
561,GACBAХH,98.0,174.0,233,-6.0,-1,92.0,2019-04-24,1.0,61.063014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,149.0,44.5
562,GACBAкЪ,116.0,182.0,326,-14.0,-1,102.0,2019-11-22,1.0,16.342466,...,13.7,72.0,5.7,4.2,4.4,20.0,25.0,7.6,146.0,42.1
563,GACBAэы,130.0,189.0,142,4.0,1,134.0,2020-11-16,2.0,23.298630,...,NaN,78.0,3.7,4.6,4.0,23.0,42.0,12.6,155.0,44.4


In [375]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Name                              565 non-null    object 
 1   WeightInit                        565 non-null    float64
 2   Height                            560 non-null    float64
 3   DateDiff                          565 non-null    int64  
 4   WeightDiff                        565 non-null    float64
 5   Target                            565 non-null    int64  
 6   TargetWeight                      565 non-null    float64
 7   StartEpizode                      565 non-null    object 
 8   ObesityDegree                     554 non-null    float64
 9   Age                               565 non-null    float64
 10  Gender                            565 non-null    int64  
 11  Prothrombin time                  315 non-null    float64
 12  Prothrom

In [376]:
# заменим NaN на средние значения
result = result.interpolate()

In [377]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Name                              565 non-null    object 
 1   WeightInit                        565 non-null    float64
 2   Height                            565 non-null    float64
 3   DateDiff                          565 non-null    int64  
 4   WeightDiff                        565 non-null    float64
 5   Target                            565 non-null    int64  
 6   TargetWeight                      565 non-null    float64
 7   StartEpizode                      565 non-null    object 
 8   ObesityDegree                     565 non-null    float64
 9   Age                               565 non-null    float64
 10  Gender                            565 non-null    int64  
 11  Prothrombin time                  565 non-null    float64
 12  Prothrom

In [378]:
result

,Name,WeightInit,Height,DateDiff,WeightDiff,Target,TargetWeight,StartEpizode,ObesityDegree,Age,...,Bilirubin total,Creatinine,Total cholesterol,Glucose (syv.),Potassium,AST,ALT,WBC- Leukocytes,HGB- Hemoglobin,HCT-Hematocrit
0,GACAAKt,100.0,176.0,32,0.0,0,100.0,2013-05-20,1.0,64.156164,...,8.90,112.0,7.40,5.60,5.00,36.0,51.0,3.90,140.0,41.70
1,GACAAXQ,100.0,172.5,358,0.0,0,100.0,2018-01-23,1.0,66.958904,...,7.00,123.0,3.10,9.30,5.70,21.0,37.0,8.20,155.0,43.40
2,GACAAZy,86.0,162.0,307,0.0,0,86.0,2009-09-04,1.0,63.602740,...,7.75,97.5,3.35,7.70,4.95,20.0,27.5,7.35,140.5,41.25
3,GACAAil,83.0,163.0,162,-3.0,-1,80.0,2013-03-21,1.0,73.065753,...,8.50,72.0,3.60,6.10,4.20,19.0,18.0,6.50,126.0,39.10
4,GACABVe,90.0,164.0,219,-1.0,-1,89.0,2009-09-16,1.0,73.452055,...,12.90,62.0,4.40,6.10,4.30,14.0,11.0,5.70,130.0,36.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,GACBAНg,100.0,168.0,317,2.0,1,102.0,2017-06-20,2.0,43.616438,...,9.30,60.0,3.70,5.70,4.70,17.0,17.0,8.80,136.8,43.70
561,GACBAХH,98.0,174.0,233,-6.0,-1,92.0,2019-04-24,1.0,61.063014,...,11.50,66.0,4.70,4.95,4.55,18.5,21.0,10.00,149.0,44.50
562,GACBAкЪ,116.0,182.0,326,-14.0,-1,102.0,2019-11-22,1.0,16.342466,...,13.70,72.0,5.70,4.20,4.40,20.0,25.0,7.60,146.0,42.10
563,GACBAэы,130.0,189.0,142,4.0,1,134.0,2020-11-16,2.0,23.298630,...,13.70,78.0,3.70,4.60,4.00,23.0,42.0,12.60,155.0,44.40


In [379]:
result.to_csv("dataset_result.csv")